In [ ]:
import numpy as np
import pandas
from matplotlib import pyplot as plt
from IPython.display import display, HTML
import scipy as sp
from scipy import stats as stats
from pprint import pprint
thisdir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/'
datadir = thisdir + 'datasets_for_analysis/'
shapefile_basedir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'
census_shapefile_tiger_basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/shapefiles/'

debug = 1

print('ok')

ok


In [3]:

df = pandas.read_csv(datadir+'reinvestment_by_census_tract_for_smallest_businesses.csv', encoding='utf-8')#, index_col=['census_tract', 'activity_year'])
[x for x in df.columns if 'mfi' in x]


['mfi', 'mfi_err', 'mfi_adj']

In [5]:
df = pandas.read_csv(datadir+'reinvestment_by_census_tract_for_smallest_businesses.csv', encoding='utf-8')#, index_col=['census_tract', 'activity_year'])

cra_info_columns = ['census_tract', 'activity_year', 'income_group_code', 'income_group', 'cra_level']

cra_loan_columns = ['census_tract', 'nLoans1', 'nLoans100k', 'nLoans250k']
cra_loan_columns += ['nLoansToSmallest', 'nLoansTotal', 'nWorkingLoans']
cra_loan_columns += ['amtLoans1_adj', 'amtLoans100k_adj', 'amtLoans250k_adj']
cra_loan_columns += ['amtLoansToSmallest_adj', 'amtLoansTotal_adj', 'amtWorkingLoans_adj']

census_columns = ['census_tract', 'pop_total', 'total_householders', 'pop_white', 'pop_black']
census_columns += ['black_householder', 'white_householder', 'owner_occ_housing_units', 'mfi']
census_columns += ['female_householder', 'hs_grad_25plus', 'unemployed_16plus', 'mfi_adj']
census_columns += ['poverty_past_12_months', 'median_home_value', 'median_year_built']
census_columns += ['pop_25plus', 'labor_force_16plus', 'poverty_status_known_last12months_total']
census_columns += [x for x in df.columns.tolist() if x[-4:] == '_pct']

census_error_columns = ['census_tract', 'pop_total_err', 'total_householders_err', 'pop_white_err', 'pop_black_err']
census_error_columns += ['black_householder_err', 'white_householder_err', 'owner_occ_housing_units_err', 'mfi_err']
census_error_columns += ['female_householder_err', 'hs_grad_25plus_serr', 'unemployed_16plus_err']
census_error_columns += ['poverty_past_12_months_err', 'median_home_value_err', 'median_year_built_err']
census_error_columns += ['pop_25plus_serr', 'labor_force_16plus_err', 'poverty_status_known_last12months_total_err']


jobs_columns = ['census_tract']
jobs_columns += [x for x in df.columns.tolist() if 'jobs' in x.lower()]


census_tract_list = df['census_tract'].dropna().drop_duplicates().sort_values().tolist()

census_tracts_df = pandas.DataFrame(census_tract_list, columns=['census_tract'])
census_tracts_df = census_tracts_df.set_index('census_tract')
census_tracts_df = census_tracts_df.assign(nYears = df.groupby('census_tract').size())
census_tracts_df = census_tracts_df.join(df.groupby('census_tract')[cra_loan_columns].sum(), how='left')

census_tracts_df = census_tracts_df.join(df[jobs_columns][df['activity_year'] == 2015].set_index('census_tract'), how='left')

census_tracts_df = census_tracts_df.join(df[census_columns][df['activity_year'] == 2017].set_index('census_tract'), how='left')
census_tracts_df = census_tracts_df.join(df[census_error_columns][df['activity_year'] == 2017].set_index('census_tract'), how='left')

#census_tracts_df[census_tracts_df['nYears'] == 7].sample(1).T
#df.columns.tolist()


#et_columns = cra_loan_columns + ['Number of jobs for workers at firms with Firm Size: 0-19 Employees']
census_tracts_df = census_tracts_df.assign(nLoans1_per_sbjob = census_tracts_df['nLoans1'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(nLoans100k_per_sbjob = census_tracts_df['nLoans100k'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(nLoans250k_per_sbjob = census_tracts_df['nLoans250k'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(nLoansToSmallest_per_sbjob = census_tracts_df['nLoansToSmallest'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(nLoansTotal_per_sbjob = census_tracts_df['nLoansTotal'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(nWorkingLoans_per_sbjob = census_tracts_df['nWorkingLoans'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(amtLoans1_adj_per_sbjob = census_tracts_df['amtLoans1_adj'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(amtLoans100k_adj_per_sbjob = census_tracts_df['amtLoans100k_adj'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(amtLoans250k_adj_per_sbjob = census_tracts_df['amtLoans250k_adj'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(amtLoansTotal_adj_per_sbjob = census_tracts_df['amtLoansTotal_adj'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df = census_tracts_df.assign(amtWorkingLoans_adj_per_sbjob = census_tracts_df['amtWorkingLoans_adj'] / census_tracts_df['Number of jobs for workers at firms with Firm Size: 0-19 Employees'])
census_tracts_df.loc[(census_tracts_df['amtWorkingLoans_adj_per_sbjob'] == np.inf), 'amtWorkingLoans_adj_per_sbjob'] = np.nan

#print('backing up...')
#census_tracts_df_bk = census_tracts_df
#print('ok')

display_census_columns = [x for x in df.columns.tolist() if x[-4:] == '_pct']
display_census_columns += ['mfi_adj']

#for tract, thisrow in census_tracts_df[(census_tracts_df['nYears'] == 7)].iterrows():
#    for thiscol in display_census_columns:
#        print('Tract {0:.2f}: Population {1:,.0f}, Loan amount ${2:,.0f}'.format(tract, thisrow[thiscol], thisrow['amtWorkingLoans_adj_per_sbjob']))

#print(census_tracts_df['pop_total'][(census_tracts_df['nYears'] == 7) & (census_tracts_df['amtWorkingLoans_adj_per_sbjob'].notnull())].values.shape)
#print(census_tracts_df['amtWorkingLoans_adj_per_sbjob'][(census_tracts_df['nYears'] == 7) & (census_tracts_df['amtWorkingLoans_adj_per_sbjob'].notnull())].values.shape)
not_significant_columns = []
for thiscol in display_census_columns:
    (r, p) = stats.pearsonr(
        census_tracts_df[thiscol][(census_tracts_df['nYears'] == 7) & (census_tracts_df['amtWorkingLoans_adj_per_sbjob'].notnull()) & (census_tracts_df[thiscol].notnull())].values, 
        census_tracts_df['amtWorkingLoans_adj_per_sbjob'][(census_tracts_df['nYears'] == 7) & (census_tracts_df['amtWorkingLoans_adj_per_sbjob'].notnull()) & (census_tracts_df[thiscol].notnull())].values
    )
    #if (p < 0.05):
    print('Correlation between {0:} and amtWorkingLoans_adj_per_sbjob (n = {1:,.0f}): {2:.3f} (p = {3:.3f})'.format(
        thiscol, 
        len(census_tracts_df[(census_tracts_df['nYears'] == 7) & (census_tracts_df['amtWorkingLoans_adj_per_sbjob'].notnull()) & (census_tracts_df[thiscol].notnull())]), 
        r, p
    ))
#    else:
#        not_significant_columns.append(thiscol)

              #census_tracts_df['amtWorkingLoans_adj_per_sbjob'][(census_tracts_df['nYears'] == 7)].values
#print('\n')
#print('Columns whose correlations with working loan amounts are not statistically significant:')
#pprint(not_significant_columns)
#for thiscol in display_census_columns:
#    
#    print(
#        stats.pearsonr(
#            census_tracts_df[thiscol][(census_tracts_df[thiscol].notnull()) & (census_tracts_df['nYears'] == 7)].tolist(),
#            census_tracts_df['amtWorkingLoans_adj_per_sbjob'][(census_tracts_df[thiscol].notnull()) & (census_tracts_df['nYears'] == 7)].tolist()
#        )
#    )
    


Correlation between pop_white_pct and amtWorkingLoans_adj_per_sbjob (n = 185): 0.123 (p = 0.096)
Correlation between pop_black_pct and amtWorkingLoans_adj_per_sbjob (n = 185): -0.129 (p = 0.080)
Correlation between white_householder_pct and amtWorkingLoans_adj_per_sbjob (n = 185): 0.123 (p = 0.095)
Correlation between black_householder_pct and amtWorkingLoans_adj_per_sbjob (n = 185): -0.134 (p = 0.069)
Correlation between female_householder_pct and amtWorkingLoans_adj_per_sbjob (n = 185): -0.023 (p = 0.754)
Correlation between hs_grad_25plus_pct and amtWorkingLoans_adj_per_sbjob (n = 185): 0.015 (p = 0.842)
Correlation between unemployed_16plus_pct and amtWorkingLoans_adj_per_sbjob (n = 185): -0.073 (p = 0.323)
Correlation between poverty_past_12_months_pct and amtWorkingLoans_adj_per_sbjob (n = 185): -0.027 (p = 0.717)
Correlation between mfi_adj and amtWorkingLoans_adj_per_sbjob (n = 183): 0.034 (p = 0.650)


In [6]:
display_census_columns

['pop_white_pct',
 'pop_black_pct',
 'white_householder_pct',
 'black_householder_pct',
 'female_householder_pct',
 'hs_grad_25plus_pct',
 'unemployed_16plus_pct',
 'poverty_past_12_months_pct',
 'mfi_adj']